In [1]:
import yaml
import requests
from bs4 import BeautifulSoup
from pymed import PubMed
import urllib.request
import requests
import gdown
import os
import pandas as pd
from PIL import Image

/usr/local/miniconda3/envs/py-env/lib/python3.11/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.2.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


### Publications

In [2]:
# url = 'https://weissmanlab.ucsf.edu/publications/publications.html'
# page = requests.get(url)

# soup = BeautifulSoup(page.text, 'html.parser')

# old_website_pubs = [(a.get('href'), a.text) for pub_div in soup.find_all(id='publication') for a in pub_div.find_all('a')]

In [2]:
pubmed = PubMed(tool="MyTool", email="lenail@mit.edu")
results = list(pubmed.query("Jonathan Weissman", max_results=500))
len(results)

280

In [3]:
pubmed_pubs = []

for result in results:
    try:
        pubmed_pubs.append(
            dict(title=result.title,
                authors=[str(author['firstname'])+' '+str(author['lastname']) for author in result.authors],
                publication_date=result.publication_date.strftime("%d/%m/%y"),
                publication_year=result.publication_date.strftime("%Y"),
                pubmed_id=result.pubmed_id.split('\n')[0],
                abstract=result.abstract,
                doi=result.doi.split('\n')[0],
                journal=result.journal))

    except:
        # print(result.toDict())

        print(result.title)
        print(result.authors)
        print(result.publication_date)
        print(result.pubmed_id)
        print(result.abstract)
        print(result.doi)
        print(result.journal)

In [4]:
replacements = {
    'bioRxiv : the preprint server for biology': 'biorxiv',
    'Methods in molecular biology (Clifton, N.J.)': 'Methods in molecular biology',
    'Current biology : CB': 'Current biology',
    'Science (New York, N.Y.)': 'Science',
    'Proceedings of the National Academy of Sciences of the United States of America': 'PNAS',
    'Molecular & cellular proteomics : MCP': 'Molecular & cellular proteomics'
}

In [5]:
for p in pubmed_pubs:
    if p['journal'] in replacements:
        p['journal'] = replacements[p['journal']]

In [6]:
pubmed_pubs = pd.DataFrame.from_records(pubmed_pubs)

In [7]:
with open('_data/publications.yml', 'r') as f:
    old_pubs = pd.json_normalize(yaml.safe_load(f))

In [8]:
old_pubs

,title,authors,publication_date,publication_year,pubmed_id,abstract,doi,journal
0,High-resolution spatial mapping of cell state ...,"[Luke W Koblan, Kathryn E Yost, Pu Zheng, Will...",24/07/25,2025,40705858,Charting the spatiotemporal dynamics of cell f...,10.1126/science.adx3800,Science
1,High-resolution spatial mapping of cell state ...,"[Luke W Koblan, Kathryn E Yost, Pu Zheng, Will...",15/07/25,2025,40661400,Charting the spatiotemporal dynamics of cell f...,10.1101/2025.06.15.659774,biorxiv
2,Shaping the composition of the mitochondrial o...,"[Gayathri Muthukumar, Jonathan S Weissman]",17/06/25,2025,40523936,Mitochondria are critical double-membraned org...,10.1038/s41556-025-01683-0,Nature cell biology
3,Perturb-Multimodal: A platform for pooled gene...,"[Reuben A Saunders, William E Allen, Xingjie P...",14/06/25,2025,40513557,Metazoan life requires the coordinated activit...,10.1016/j.cell.2025.05.022,Cell
4,Transcriptome-wide analysis of differential ex...,"[Ajay Nadig, Joseph M Replogle, Angela N Pogso...",22/04/25,2025,40259084,Single-cell CRISPR screens such as Perturb-seq...,10.1038/s41588-025-02169-3,Nature genetics
...,...,...,...,...,...,...,...,...
274,Generation of prion transmission barriers by m...,"[Peter Chien, Angela H DePace, Sean R Collins,...",22/08/03,2003,12931190,Self-propagating beta-sheet-rich protein aggre...,10.1038/nature01894,Nature
275,Directed evolution of substrate-optimized GroE...,"[Jue D Wang, Christophe Herman, Kimberly A Tip...",01/01/03,2003,12507429,GroEL/S chaperonin ring complexes fold many un...,10.1016/s0092-8674(02)01198-4,Cell
276,The FAD- and O(2)-dependent reaction cycle of ...,"[Benjamin P Tu, Jonathan S Weissman]",28/11/02,2002,12453408,The endoplasmic reticulum (ER) supports disulf...,10.1016/s1097-2765(02)00696-2,Molecular cell
277,Origins and kinetic consequences of diversity ...,"[Angela H DePace, Jonathan S Weissman]",09/04/02,2002,11938354,A remarkable feature of prions is that infecti...,10.1038/nsb786,Nature structural biology


#### New DOIs:

In [9]:
pubmed_pubs[ ~pubmed_pubs.doi.isin(old_pubs.doi)]


,title,authors,publication_date,publication_year,pubmed_id,abstract,doi,journal
19,Author Correction: CRISPR technologies for gen...,"[Lukas Villiger, Julia Joung, Luke Koblan, Jon...",14/05/24,2024,38740926,None,10.1038/s41580-024-00745-9,Nature reviews. Molecular cell biology


remove dois

In [10]:
remove_pubs = ["37853259","38740926"] # author correction
pubmed_pubs = pubmed_pubs[ ~pubmed_pubs.pubmed_id.isin(remove_pubs)]

#### Updates to existing IDs:

In [11]:
pubmed_pubs['update'] = 'current'
old_pubs['update'] = 'previous'

df = pd.concat([pubmed_pubs[pubmed_pubs.doi.isin(old_pubs.doi)],old_pubs[old_pubs.doi.isin(pubmed_pubs.doi)]]).reset_index(drop=True)
df.loc[df.astype(str).drop_duplicates(subset=['title', 'authors', 'publication_date', 
                                             'publication_year', 'pubmed_id', 'abstract', 'doi', 'journal'], keep=False).index].sort_values(by='doi')

,title,authors,publication_date,publication_year,pubmed_id,abstract,doi,journal,update
109,Combinatorial genetics in liver repopulation a...,"[Kirk J Wangensteen, Yue J Wang, Zhixun Dou, A...",02/11/17,2017,29091290,Clustered regularly interspaced short palindro...,10.1002/hep.29626,"Hepatology (Baltimore, Md.)",current
387,Combinatorial genetics in liver repopulation a...,"[Kirk J Wangensteen, Yue J Wang, Zhixun Dou, A...",02/11/17,2017,29091290,Clustered regularly interspaced short palindro...,10.1002/hep.29626,Hepatology,previous
416,Targeting the AAA ATPase p97 as an Approach to...,"[Daniel J Anderson, Ronan Le Moigne, Stevan Dj...",12/11/15,2015,26555175,p97 is a AAA-ATPase with multiple cellular fun...,10.1016/j.ccell.2015.10.002,Cancer cell,previous
138,Targeting the AAA ATPase p97 as an Approach to...,"[Daniel J Anderson, Ronan Le Moigne, Stevan Dj...",12/11/15,2015,26555175,p97 is a AAA-ATPase with multiple cellular fun...,10.1016/j.ccell.2015.10.002,Cancer cell,current
460,Ribosome profiling provides evidence that larg...,"[Mitchell Guttman, Pamela Russell, Nicholas T ...",03/07/13,2013,23810193,Large noncoding RNAs are emerging as an import...,10.1016/j.cell.2013.06.009,Cell,previous
...,...,...,...,...,...,...,...,...,...
422,Pharmacological dimerization and activation of...,"[Carmela Sidrauski, Jordan C Tsai, Martin Kamp...",16/04/15,2015,25875391,The general translation initiation factor eIF2...,10.7554/eLife.07314,eLife,previous
140,Paradoxical resistance of multiple myeloma to ...,"[Diego Acosta-Alvear, Min Y Cho, Thomas Wild, ...",04/09/15,2015,26327694,"Hallmarks of cancer, including rapid growth an...",10.7554/eLife.08153,eLife,current
418,Paradoxical resistance of multiple myeloma to ...,"[Diego Acosta-Alvear, Min Y Cho, Thomas Wild, ...",04/09/15,2015,26327694,"Hallmarks of cancer, including rapid growth an...",10.7554/eLife.08153,eLife,previous
108,The Human Cell Atlas.,"[Aviv Regev, Sarah A Teichmann, Eric S Lander,...",06/12/17,2017,29206104,The recent advent of methods for high-throughp...,10.7554/eLife.27041,eLife,current


#### Removal of IDs:

In [12]:
old_pubs[ ~old_pubs.doi.isin(pubmed_pubs.doi)]

,title,authors,publication_date,publication_year,pubmed_id,abstract,doi,journal,update
24,Author Correction: Efficient C•G-to-G•C base e...,"[Luke W Koblan, Mandana Arbab, Max W Shen, Jef...",19/10/23,2023,37853259,None,10.1038/s41587-023-02028-8,Nature biotechnology,previous


#### For now: 

In [13]:
pubmed_pubs = pd.concat([pubmed_pubs[ ~pubmed_pubs.doi.isin(old_pubs.doi)], old_pubs])

pubmed_pubs = pubmed_pubs.drop('update', axis=1)
pubmed_pubs = pubmed_pubs.sort_values(["publication_year","pubmed_id"],ascending=False).reset_index(drop=True)
pubs_records = [{k:v for k,v in m.items() if str(v) != 'nan'} for m in pubmed_pubs.to_dict(orient='records')]


In [14]:
pubs_records

[{'title': 'High-resolution spatial mapping of cell state and lineage dynamics in vivo with PEtracer.',
  'authors': ['Luke W Koblan',
   'Kathryn E Yost',
   'Pu Zheng',
   'William N Colgan',
   'Matthew G Jones',
   'Dian Yang',
   'Arhan Kumar',
   'Jaspreet Sandhu',
   'Alexandra Schnell',
   'Dawei Sun',
   'Can Ergen',
   'Reuben A Saunders',
   'Xiaowei Zhuang',
   'William E Allen',
   'Nir Yosef',
   'Jonathan S Weissman'],
  'publication_date': '24/07/25',
  'publication_year': '2025',
  'pubmed_id': '40705858',
  'abstract': 'Charting the spatiotemporal dynamics of cell fate determination in development and disease is a long-standing objective in biology. Here we present the design, development, and extensive validation of PEtracer, a prime editing-based, evolving lineage tracing technology compatible with both single-cell sequencing and multimodal imaging methodologies to jointly profile cell state and lineage in dissociated cells or while preserving cellular context in ti

In [15]:
open('_data/publications.yml', 'w').write(yaml.dump(pubs_records, default_flow_style=False, sort_keys=False))

438632

### Trim down lab photos to max size

In [16]:
import os
for img_path in os.listdir('assets/img/people/'):
    try:
        image = Image.open('assets/img/people/'+img_path)
        if image.size[0] > 400 or image.size[1] > 400:
            image.thumbnail((400, 400))
            # convert image to jpg if png
            if image.format == 'PNG':
                image = image.convert('RGB')
                img_path = img_path.split('.')[0]+'.jpg'
            image.save('assets/img/people/'+img_path)
    except:
        print(img_path)

.DS_Store


In [15]:
# id must be both the publication first/last, and the name of the image. Name is just the display name.

## Update people page

In [17]:
url = 'https://docs.google.com/spreadsheets/d/1-Eju9h1XovEBoBv0DGpxh92GYsZ8bzGkxRdRgUb7hvg'
people = pd.read_csv(url+'/export?format=csv')
# strip strings
df_obj = people.select_dtypes(['object'])
people[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [18]:
with open('_data/people.yml', 'r') as f:
    old_people_data = pd.json_normalize(yaml.safe_load(f))

#### New IDs:

In [19]:
people[ ~people.id.isin(old_people_data.id)]

,name,pos,email,id,alum,now
2,Tessa Dinnie,Administrative Lab Manager,tdinnie@wi.mit.edu,TessaDinnie,False,NaN
3,Irina Cashen,Administrative Assistant,icashen@wi.mit.edu,IrinaCashen,False,NaN
35,Yijia Chen,Grad student,ychen@wi.mit.edu,YijiaChen,False,NaN
36,Max Hyman,Technician,mhyman@wi.mit.edu,MaxHyman,False,NaN
61,Xinjie Pan,Visiting Scientist,xpan@wi.mit.edu,XinjiePan,False,NaN
63,Gokul Gowri,Visiting Scientist,gokulg@mit.edu,GokulGowri,False,NaN


#### Updates to existing IDs:

In [20]:
people['update'] = 'current'
old_people_data['update'] = 'previous'

pd.concat([people[people.id.isin(old_people_data.id)],old_people_data[old_people_data.id.isin(people.id)]]).drop_duplicates(subset=['name', 'pos', 'email', 'alum', 'now'], keep=False).sort_values(by='id')


,name,pos,email,id,alum,now,update
26,Edwin Neumann,Post-doc,enn@wi.mit.edu,EdwinNeumann,False,NaN,current
31,Edwin Neumann,Grad student,enn@wi.mit.edu,EdwinNeumann,False,NaN,previous
51,Ever Marinelli,Undergrad,emarinelli@wi.mit.edu,EverMarinelli,True,NaN,current
48,Ever Marinelli,Undergrad,emarinelli@wi.mit.edu,EverMarinelli,False,NaN,previous
30,Gayathri Muthukumar,Grad student,gmkumar@wi.mit.edu,GayathriMuthukumar,True,"Postdoctoral Fellow | Bertozzi Lab, ChEM-H Ins...",current
28,Gayathri Muthukumar,Grad student,gmkumar@wi.mit.edu,GayathriMuthukumar,False,NaN,previous
52,Jiahe Ai,Undergrad,jaipynnic@wi.mit.edu,JiaheAi,True,PhD student | Cancer Science Institute of Si...,current
49,Jiahe Ai,Undergrad,jaipynnic@wi.mit.edu,JiaheAi,False,NaN,previous
62,Reuben Saunders,Visiting Scientist,reubens@wi.mit.edu,ReubenSaunders,False,NaN,current
25,Reuben Saunders,Grad student,reubens@wi.mit.edu,ReubenSaunders,True,Junior Fellow | Harvard Society of Fellows,previous


#### Removal of IDs:

In [21]:
old_people_data[ ~old_people_data.id.isin(people.id)]

,name,pos,email,id,alum,now,update


#### Write updated sheet to yaml

In [22]:
people = people.drop('update', axis=1)
people_records = [{k:v for k,v in m.items() if pd.notnull(v)} for m in people.to_dict(orient='records')]


In [23]:
class MyDumper(yaml.SafeDumper):
    # HACK: insert blank lines between top-level objects
    # inspired by https://stackoverflow.com/a/44284819/3786245
    def write_line_break(self, data=None):
        super().write_line_break(data)

        if len(self.indents) == 1:
            super().write_line_break()

In [24]:
open('_data/people.yml', 'w').write(yaml.dump(people_records, Dumper=MyDumper, default_flow_style=False, sort_keys=False))


21594